In [ ]:
"""
You need to run this cell for the code in following cells to work.
"""

# Enable module reloading
%load_ext autoreload
%autoreload 2

# Enable interactive plots
%matplotlib notebook

import sys
sys.path.append('..')


# Week 3

__Goals for this week__



__How to solve this lab?__

Just follow this notebook and run all the code. When prompted complete the code in the referenced files. Correct answers for some exercises are provided at the bottom.

__Feedback__

This lab is a work in progress. If you notice a mistake, notify us or you can even make a pull request. Also please fill the questionnaire after you finish this lab to give us feedback.


## Artificial Neuron

Vysvetlit

Dokodit do week2.model predict a compute gradients

Show at simple logistic function data

## Multilayer Perceptron

layers
activation functions - why do we need them?

dense layers, feedforward layers (part of bigger networks)

Neural network playground

PA: forward pass, derivatives - SGD will be reused from week 2

## Classification

softmax with crossentropy loss function

why it works - further reading

MNIST data?


further reading
NNandDL online book - backprop
COlah - backprop
softmax?

# Week 4

## Vectorization

Change MLP so it works with matrices

## ML Experiment

train / dev / test split
bias and variance
additional data - house pricing?
??

## Regularization

Exercises or for project?
Final part of low level coding - maybe a little bit easier so they can experiment with their models and fine tune them?

## Training algorithms
ruder outline?
Adam explained, other just referenced

They can start working on 1st project after this

# Project (Deadline W6)

1. dynamic multilayer
2. early stopping
3. regularization: 2 of following:
    - l1 or l2 or weight decay
    - dropout
    - dropconnect
    - batch normalization
    - after consulting - any other
4. training algorithm: 1 of following:
    - adam
    - rmsprop
    - adagrad
    - after consulting - any other

## Key Concepts from This Week

## Further Reading

## Sources

## Correct Answers